In [2]:
import sqlite3
import pandas as pd

# Load your dataset
df = pd.read_csv('videos_with_sentiment.csv')

# Connect to SQLite (creates a new DB file)
conn = sqlite3.connect('youtube_trending.db')

# Save dataframe as a table
df.to_sql('trending_videos', conn, if_exists='replace', index=False)

print("✅ Data loaded into SQLite DB!")


✅ Data loaded into SQLite DB!


In [3]:
query = """
SELECT category_id, AVG(views) AS avg_views
FROM trending_videos
GROUP BY category_id
ORDER BY avg_views DESC
LIMIT 5;
"""

result_df = pd.read_sql_query(query, conn)
print(result_df)


   category_id     avg_views
0           10  9.194677e+06
1           30  3.065001e+06
2            1  2.962616e+06
3           29  2.096646e+06
4           17  1.896538e+06


In [6]:
query = """
SELECT category_id, AVG(likes) AS avg_likes
FROM trending_videos
GROUP BY category_id
ORDER BY avg_likes DESC
LIMIT 5;
"""

result_df = pd.read_sql_query(query, conn)
print(result_df)


   category_id      avg_likes
0           10  224803.195981
1           29  156691.960317
2           23   61261.602938
3            1   58154.659878
4           20   54613.103706


In [9]:
query = """
SELECT category_id, AVG(trending_date) AS avg_day_trending
FROM trending_videos
GROUP BY category_id
ORDER BY avg_day_trending DESC
LIMIT 5;
"""

result_df = pd.read_sql_query(query, conn)
print(result_df)


   category_id  avg_day_trending
0           30         18.067500
1           15         17.991297
2           10         17.969527
3            1         17.945176
4           29         17.943135


In [10]:
print(df.columns)

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description', 'country', 'title_sentiment',
       'tag_sentiment', 'title_sentiment_label', 'tag_sentiment_label'],
      dtype='object')


In [11]:
query = """
SELECT country, title_sentiment_label, COUNT(*) AS count
FROM trending_videos
GROUP BY country, title_sentiment_label
ORDER BY country, count DESC;
"""

result_df = pd.read_sql_query(query, conn)
print(result_df)


  country title_sentiment_label  count
0      IN               Neutral  25324
1      IN              Positive   7592
2      IN              Negative   4436
3      UK               Neutral  22168
4      UK              Positive   9034
5      UK              Negative   7714
6      US               Neutral  22680
7      US              Positive  10521
8      US              Negative   7748


In [12]:
result_df.to_csv('top_categories_avg_views.csv', index=False)


In [13]:
df['trending_date'] = pd.to_datetime(df['trending_date'], errors='coerce')


C:\Users\Sayan Saha\AppData\Local\Temp\ipykernel_22248\716050543.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['trending_date'] = pd.to_datetime(df['trending_date'], errors='coerce')


In [14]:
# Group by video_id and country (to handle each country separately)
duration_df = df.groupby(['video_id', 'country'])['trending_date'].nunique().reset_index()

# Rename column
duration_df = duration_df.rename(columns={'trending_date': 'trending_duration'})

# Merge back into main dataframe
df = pd.merge(df, duration_df, on=['video_id', 'country'], how='left')


In [15]:
df.to_csv('final_video_duration.csv', index=False)
print("✅ Trending duration column added and file saved as 'merged_trending_videos_with_duration.csv'!")


✅ Trending duration column added and file saved as 'merged_trending_videos_with_duration.csv'!
